In [84]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [85]:
disasterCostData = "../ml/data/costDataRevised.csv"
cost_df = pd.read_csv(disasterCostData)
cost_df.head()

,Unnamed: 0,disasterNumber,obligationAmount,state,incidentType,declarationTitle,incidentBeginDate,lat,lon,incidentDate,incidentTime
0,0,3345,904490.14,WV,Severe Storms,SEVERE STORMS,2012-06-29T22:30:00.000Z,39.453524,-78.039552,2012-06-29,22:30:00.000Z
1,1,3346,842464.67,OH,Severe Storms,SEVERE STORMS,2012-06-29T22:00:00.000Z,38.839894,-83.505170,2012-06-29,22:00:00.000Z
2,2,3347,1409793.92,LA,Hurricane,TROPICAL STORM ISAAC,2012-08-26T00:00:00.000Z,30.211644,-90.918610,2012-08-26,00:00:00.000Z
3,3,3348,253189.73,MS,Hurricane,TROPICAL STORM ISAAC,2012-08-26T00:00:00.000Z,33.077787,-89.568066,2012-08-26,00:00:00.000Z
4,4,3349,0.00,MD,Hurricane,HURRICANE SANDY,2012-10-26T15:36:00.000Z,38.528853,-76.537764,2012-10-26,15:36:00.000Z


In [91]:
weather_data = []
for index, row in cost_df.iterrows():
    lat = row['lat'] 
    lon = row['lon']
    date = row['incidentDate']
    #lat = 39.4535243
    #lon = -78.0395522
    #date = '2012-06-29'
    url = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat}%2C{lon}/{date}?unitGroup=us&key=WAH7SGPNNNY6MRXZ7KYSSKC8M&include=obs'
    response = requests.get(url)
    weather_data.append(response.json())


In [92]:
weather_data[0]

{'queryCost': 1,
 'latitude': 39.4535243,
 'longitude': -78.0395522,
 'resolvedAddress': '39.4535243,-78.0395522',
 'address': '39.4535243,-78.0395522',
 'timezone': 'America/New_York',
 'tzoffset': -4.0,
 'days': [{'datetime': '2012-06-29',
   'datetimeEpoch': 1340942400,
   'tempmax': 102.0,
   'tempmin': 71.4,
   'temp': 86.7,
   'feelslikemax': 106.9,
   'feelslikemin': 71.4,
   'feelslike': 89.8,
   'dew': 66.7,
   'humidity': 54.49,
   'precip': 0.09,
   'precipprob': None,
   'precipcover': 8.33,
   'preciptype': None,
   'snow': None,
   'snowdepth': None,
   'windgust': 47.9,
   'windspeed': 22.8,
   'winddir': 269.8,
   'pressure': 1007.2,
   'cloudcover': 12.8,
   'visibility': 9.6,
   'solarradiation': None,
   'solarenergy': None,
   'uvindex': 0.0,
   'sunrise': '05:48:08',
   'sunriseEpoch': 1340963288,
   'sunset': '20:43:13',
   'sunsetEpoch': 1341016993,
   'moonphase': 0.38,
   'conditions': 'Rain',
   'description': 'Clear conditions throughout the day with late aft

In [95]:
df = pd.json_normalize(
    weather_data, 
    record_path =['days'], 
    meta=['latitude','longitude','resolvedAddress']
)

In [96]:
df.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunsetEpoch,moonphase,conditions,description,icon,stations,source,latitude,longitude,resolvedAddress
0,2012-06-29,1340942400,102.0,71.4,86.7,106.9,71.4,89.8,66.7,54.49,...,1341016993,0.38,Rain,Clear conditions throughout the day with late ...,rain,"[KMRB, 72405303717, 72406693706, 72417713734, ...",obs,39.453524,-78.039552,"39.4535243,-78.0395522"
1,2012-06-29,1340942400,98.7,68.0,81.9,107.5,68.0,84.7,66.7,62.01,...,1341018190,0.38,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"[72429613841, KILN]",obs,38.839894,-83.50517,"38.8398935,-83.5051704"
2,2012-08-26,1345957200,91.6,72.9,80.9,97.1,72.9,83.4,71.9,76.99,...,1346027565,0.36,Clear,Partly cloudy throughout the day.,clear-day,"[72231203908, KBTR, 72232013970, 72231012916, ...",obs,30.211644,-90.91861,"30.2116442,-90.9186104"
3,2012-08-26,1345957200,89.2,68.1,77.9,94.0,68.1,79.8,69.8,78.47,...,1346027417,0.36,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KGWO, KSTF, 74758013978]",obs,33.077787,-89.568066,"33.0777868,-89.5680661"
4,2012-10-26,1351224000,70.6,61.8,64.9,70.6,61.8,64.9,59.6,83.24,...,1351289562,0.45,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[72404013721, KNHK, KADW, 74594013705]",obs,38.528853,-76.537764,"38.5288529,-76.537764"


In [107]:
disaster = cost_df['disasterNumber'].to_list()

In [109]:
df['disasterNumber'] = disaster

In [110]:
df.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,moonphase,conditions,description,icon,stations,source,latitude,longitude,resolvedAddress,disasterNumber
0,2012-06-29,1340942400,102.0,71.4,86.7,106.9,71.4,89.8,66.7,54.49,...,0.38,Rain,Clear conditions throughout the day with late ...,rain,"[KMRB, 72405303717, 72406693706, 72417713734, ...",obs,39.453524,-78.039552,"39.4535243,-78.0395522",3345
1,2012-06-29,1340942400,98.7,68.0,81.9,107.5,68.0,84.7,66.7,62.01,...,0.38,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"[72429613841, KILN]",obs,38.839894,-83.50517,"38.8398935,-83.5051704",3346
2,2012-08-26,1345957200,91.6,72.9,80.9,97.1,72.9,83.4,71.9,76.99,...,0.36,Clear,Partly cloudy throughout the day.,clear-day,"[72231203908, KBTR, 72232013970, 72231012916, ...",obs,30.211644,-90.91861,"30.2116442,-90.9186104",3347
3,2012-08-26,1345957200,89.2,68.1,77.9,94.0,68.1,79.8,69.8,78.47,...,0.36,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[KGWO, KSTF, 74758013978]",obs,33.077787,-89.568066,"33.0777868,-89.5680661",3348
4,2012-10-26,1351224000,70.6,61.8,64.9,70.6,61.8,64.9,59.6,83.24,...,0.45,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[72404013721, KNHK, KADW, 74594013705]",obs,38.528853,-76.537764,"38.5288529,-76.537764",3349


In [111]:
merged_weather_data = pd.merge(df, cost_df, how='inner',on='disasterNumber')

In [117]:
merged_weather_data_cleanded = merged_weather_data.drop(columns=['precipprob','preciptype','solarradiation','solarenergy','uvindex','Unnamed: 0'])

In [118]:
merged_weather_data.to_csv('../ml/data/disasterWeather.csv')